# Análisis de sentimientos en novelas clásicas

Realizado por Pablo Aguilera Onieva

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
import requests
from bs4 import BeautifulSoup

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from collections import Counter
import plotly.graph_objects as go

In [ ]:
nltk.download("wordnet")
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
wordnet_to_penn = {
 'n': 'NN', # sustantivo
 'v': 'VB', # verbo
 'a': 'JJ', # adjetivo
 's': 'JJ', # adjetivo superlativo
 'r': 'RB', # adverbio
 'c': 'CC' # conjunción
}

penn_to_wordnet = {
 'CC': 'c', # Coordinating conjunction
 'CD': 'c', # Cardinal number
 'DT': 'c', # Determiner
 'EX': 'c', # Existential there
 'FW': 'x', # Foreign word
 'IN': 'c', # Preposition or subordinating conjunction
 'JJ': 'a', # Adjective
 'JJR': 'a', # Adjective, comparative
 'JJS': 'a', # Adjective, superlative
 'LS': 'c', # List item marker
 'MD': 'v', # Modal
 'NN': 'n', # Noun, singular or mass
 'NNS': 'n', # Noun, plural
 'NNP': 'n', # Proper noun, singular
 'NNPS': 'n', # Proper noun, plural
 'PDT': 'c', # Predeterminer
 'POS': 'c', # Possessive ending
 'PRP': 'n', # Personal pronoun
 'PRP$': 'n', # Possessive pronoun
 'RB': 'r', # Adverb
 'RBR': 'r', # Adverb, comparative
 'RBS': 'r', # Adverb, superlative
 'RP': 'r', # Particle
 'SYM': 'x', # Symbol
 'TO': 'c', # to
 'UH': 'x', # Interjection
 'VB': 'v', # Verb, base form
 'VBD': 'v', # Verb, past tense
 'VBG': 'v', # Verb, gerund or present participle
 'VBN': 'v', # Verb, past participle
 'VBP': 'v', # Verb, non-3rd person singular present
 'VBZ': 'v', # Verb, 3rd person singular present
 'WDT': 'c', # Wh-determiner
 'WP': 'n', # Wh-pronoun
 'WP$': 'n', # Possessive wh-pronoun
 'WRB': 'r', # Wh-adverb
 'X': 'x' # Any word not categorized by the other tags
 }

1. Cargar en una estructura de datos Python el léxico de sentimientos NRC (National
Research Council). Se puede descargar el léxico de sentimientos NRC desde el siguiente enlace:
https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm. Asegurarse de entender cómo se
estructura el léxico y cómo se mapean las palabras a las emociones. Tener en cuenta que hay varios
ficheros con la misma información: un fichero con toda la información, un fichero por emoción, etc.
Elegir la opción que se estime oportuna. Considerar cómo organizar el léxico en memoria para un acceso
rápido durante el análisis.


Para esta sección se ha descargado un fichero zip del enlace del enunciado. Este fichero está compuesto por diferentes ficheros, de lo cuales se va a utilizar el nombrado como "Wordlevel". Este fichero contiene por cada palabra un valor indicando si un sentimiento o emoción esta asociado o no a esa palabra (1 o 0).

Los sentimientos estan divididos en dos tipos, positive (positivo) y negative (negativo) y las emociones estan divididas en anger (ira), anticipation (anticipación), disgust (disgusto), fear (miedo), joy (alegría), sadness (tristeza), surprise (sorpresa) y trust (confianza). Para el caso de estudio solo interesa la sección de las emociones, por ello se elimina los casos que tengan positivo y negativo. También se transforma el DataFrame "emociones_ingles" en un diccionario para un acceso rápido durante el análisis.

Este diccionario esta formado con las palabras como claves, y como valores listas que contienen las emociones asociadas a cada una de las claves. En caso de que una palabra no tenga una emoción asociada, simplemente tiene como valor una lista vacia.

In [ ]:
filepath = "NRC-Emotion-Lexicon-Wordlevel-v0.92.txt"
emociones_ingles = pd.read_csv(filepath,  names=["word", "emotion", "association"], sep='\t')
emociones_ingles = emociones_ingles[(emociones_ingles['emotion'] != 'positive') & (emociones_ingles['emotion'] != 'negative')].reset_index()
del emociones_ingles["index"]

lista_emociones = emociones_ingles["emotion"].unique()
emociones_ingles.head(12)

,word,emotion,association
0,aback,anger,0
1,aback,anticipation,0
2,aback,disgust,0
3,aback,fear,0
4,aback,joy,0
5,aback,sadness,0
6,aback,surprise,0
7,aback,trust,0
8,abacus,anger,0
9,abacus,anticipation,0


En esta sección se aprovecha para añadir la pos-tag a cada una de las palabras, lo cual debería de ser en la siguiente parte. Un problema que se ha encontrado es que algunas palabras del lexicón generado por NRC no se encuentran contenidas en Wordnet o necesitan de un preprocesado anterior, por lo que no se puede obtener su pos-tag a partir de los synsets de Wordnet, algunos ejemplos de esto son las palabras abba, abovementioned, affiche, accueil o airbag. Por esta razón, se decide utilizar la función nltk.pos_tag() para clasificarlas de manera inicial, ya que ha sido entrenado con otro tipos de corpus en vez de utilizar especificamente Wordnet.
Esto es debido a que no son palabras comunes, o son de otro idioma pero tienen cierto uso en algunos casos.

In [ ]:
emociones_dict = {}

# Iterar sobre las filas del DataFrame
for index, row in emociones_ingles.iterrows():
    if row['association'] == 0:
        emociones_dict.setdefault((str(row['word']),pos_tag([str(row["word"]).lower()])[0][1]), [])
    else:
        emociones_dict.setdefault((str(row['word']),pos_tag([str(row["word"]).lower()])[0][1]), []).append(row['emotion'])


2. Extender el léxico NRC utilizando WordNet (https://www.nltk.org/howto/wordnet.html)
para incluir sinónimos, hipónimos e hiperónimos de las palabras ya presentes en el léxico. También
puedes usar la función derivationally_related_forms() de WordNet de modo que el léxico pueda
extenderse más aún. Esta función devuelve una lista de formas derivadas de una palabra, como plurales,
participios pasados, etc. Esto puede ser útil para encontrar variaciones de una palabra que puedan estar
asociadas con la misma emoción. El léxico deberá implementarse como un diccionario Python que
tenga como clave una dupla <lemma, POS-tag>, y como valor la lista de emociones con las que dicha
dupla se podría asociar.

Para obtener los sinónimos se ha realizado a través de los synsets de Wordnet. Un synset es una agrupación de palabras que comparten un significado similar. Cada synset esta asocioado a una palabra y una lista de palabras sinónimas que comparten el mismo significado. Estos sinónimos son lemas, es decir, la forma canónica de una palabra.

Para los sinónimos se ha mantenido el formato utilizado en el léxico "emociones_dict" y se ha tenido en cuentas las siguientes casuísticas:
- Los sinónimos no se pueden repetir entre ellos, salvo que su pos_tag sea distinto. Esto último quiere decir que una misma palabra como puede ser "aback" sería permitida si fuese un sustantivo y un adverbio, pero no como dos sutantivos.
- Al obtener los sinónimos se ha encontrado con incosistencias respecto al léxico de NRC. Para entenderlo mejor se utiliza la palabra "abandon" como ejemplo. La palabra "abandon" tiene como sinónimos a las palabras "desertion" y "defection". Estos sinónimos ya estan contenido en el léxico de NRC, pero al comparar las emociones de la palabra "abandonment" con esos sinónimos en NRC, se obtiene resultados distintos. Esto a que los sinónimos perfectos no son tan comunes, por lo que puede que las acepciones específicas de estos sinónimos llevan a emociones distintas. Por ello se ha tomado la decisión de darle prioridad a las emociones contenidas en el léxico NRC, al tener unos datos más contrastados.
- Si el sinónimo no esta contenido en "emociones_dict", se ha decidido añadirle las mismas emociones de la palabra asociada con ese sinónimo. Esto es debido a que comparten cierta coherencia en el contexto emocional, aunque puedan tener acepciones específicas.

In [ ]:
sinononimos={}
for palabra in emociones_dict:
    for synset in wn.synsets(palabra[0]):
        if(synset):
            for lemma in synset.lemmas():
                sinonimo=(lemma.name(),wordnet_to_penn[synset.pos()])
                if (sinonimo not in sinononimos and sinonimo not in emociones_dict):
                    sinononimos[sinonimo]=emociones_dict[palabra]


Un hiperónimo es una palabra que tiene un significado más general y abarca conjunto más amplio de palabras. Como podría ser la palabra "comida" que agrupa palabras como "manzana", "pera", etc.

En cuanto a los hiperónimos, se procesa los hiperónimos por synset de una palabra. Se mantiene el mismo formato de los diccionarios "emociones_dict" y "sinónimos" <lemma, pos_tag>. Se mantiene las mismas casuísticas pero añadiendo que no se pueden repetir con los elementos del diccionario "sinónimos". También para la selección de las emociones en este caso se ha decidido tomar las emociones de los sinónimos del hiperónimo y juntar todas sus emociones sin repeticiones. Aunque no es una aproximación exacta, es probable que los sinónimos tengan emociones relacionadas. En caso de que el hiperónimo no pertenezca al léxico, se le añadiran las emociones de la palabra con la que se obtiene ese hiperónimo.

In [ ]:
def obtener_emociones_hiperonimo(palabra,hiperonimo):
    emociones=[]
    for lema in hiperonimo.lemmas():
        hiperonimo_aux=(lema.name(),wordnet_to_penn[hiperonimo.pos()])
        if(hiperonimo_aux in emociones_dict):
            emociones=emociones+list(set(emociones_dict[hiperonimo_aux]))
        else: emociones=emociones_dict[palabra]
    return hiperonimo_aux,emociones

In [ ]:
hiperonimos={}

for palabra in emociones_dict:
    for synset in wn.synsets(palabra[0]):
        if(synset):
            for hiperonimo in synset.hypernyms():
                if (hiperonimo):
                    hiperonimo_aux,emociones=obtener_emociones_hiperonimo(palabra,hiperonimo)
                    if(hiperonimo_aux not in hiperonimos and hiperonimo_aux not in emociones_dict
                       and hiperonimo_aux not in sinononimos):
                        hiperonimos[hiperonimo_aux]=emociones

Un hipónimo es una palabra que tiene un significado más específico y se encuentra dentro de una categoría más amplia, su hiperónimo.

Se ha mantenido las mismas consideraciones que en los anteriores casos, pero añadiendo que no haya repeticiones rescpecto al diccionario de hiperónimos. Para las emociones asociadas al hipónimo, en caso de no estar contenido en ninguno de los diccionarios, se ha decidido que tome como valor las emociones de su hiperónima al considerar que tienen una relación sus contextos emocionales.

In [ ]:
hiponimos={}
for palabra in emociones_dict:
    for synset in wn.synsets(palabra[0]):
        if(synset):
            for hiponimo in synset.hyponyms():
                if (hiponimo):
                    for lema in hiponimo.lemmas():
                        hiponimo_aux = (lema.name(),wordnet_to_penn[hiponimo.pos()])
                        if(hiponimo_aux not in hiponimos and hiponimo_aux not in emociones_dict
                            and hiponimo_aux not in sinononimos and hiponimo_aux not in hiperonimos):
                            hiponimos[hiponimo_aux]=emociones_dict[palabra]

También se ha utilizado la funcióbn "derivationally_related_forms" de Wordnet para encontrar distintas formas de palabras que tengan las mismas emociones. Esto sirve para obtener un léxico más completo.

In [ ]:
derivados={}

for palabra in emociones_dict:
    for synset in wn.synsets(palabra[0]):
        if(synset):
            for lemma in synset.lemmas():
                if(lemma):
                    for derivado in lemma.derivationally_related_forms():
                        if(derivado):
                            name=derivado.name()
                            derivado_aux=(name,pos_tag([name])[0][1])
                            if(derivado_aux not in emociones_dict and derivado_aux not in sinononimos
                               and derivado_aux not in hiperonimos and derivado_aux not in hiponimos
                               and derivado_aux not in derivados):
                                derivados[derivado_aux]=emociones_dict[palabra]

Y con todos estos diccionarios procesados de diferentes características del léxico NRC, los añadimo al léxico "emociones_dict".

In [ ]:
emociones_dict.update(sinononimos)
emociones_dict.update(hiperonimos)
emociones_dict.update(hiponimos)
emociones_dict.update(derivados)

Se ha observado que se añaden palabras y números, que al obtenerlos a través de los hipónimos de alguna palabra comparten la emoción de alguna acepción de la palabra evaluada. Por ejemplo, la palabra "letter" tiene como emoción "anticipation" y un grupo de hipónimos de "letter" son las letras del abecedario, las cuales no tienen ninguna emoción. Entonces para limpiar el léxico un poco se presupone que las palabras que tienen longitud 1 o son números no tienen ninguna emoción.

In [ ]:
for palabra, emocion in emociones_dict.items():
    if len(palabra[0]) == 1 or palabra[0].isdigit():
        emociones_dict[palabra] = []

3. Cargar el texto de novelas clásicas disponibles en Project Gutenberg. Utilizar la biblioteca
Beautiful Soup para extraer el texto de las páginas HTML y prepararlo para el análisis.
El siguiente es un diccionario con 10 novelas conocidas que están accesibles en Project Gutenberg que
puedes usar en tu código.

In [ ]:

books = {
    'Moby Dick': 'https://www.gutenberg.org/cache/epub/2701/pg2701-images.html',

    'War and Peace': 'https://www.gutenberg.org/cache/epub/2600/pg2600-images.html',

    'Pride and Prejudice': 'https://www.gutenberg.org/cache/epub/1342/pg1342-images.html',

    'Crime and Punishment': 'https://www.gutenberg.org/cache/epub/2554/pg2554-images.html',

    'The Adventures of Sherlock Holmes': 'https://www.gutenberg.org/cache/epub/1661/pg1661-images.html',

    'Ulysses': 'https://www.gutenberg.org/cache/epub/4300/pg4300-images.html',

    'The Odyssey': 'https://www.gutenberg.org/cache/epub/1727/pg1727-images.html',

    'The Divine Comedy': 'https://www.gutenberg.org/cache/epub/8800/pg8800-images.html',

    'The Great Gatsby': "https://www.gutenberg.org/cache/epub/64317/pg64317-images.html",

    'Critias': 'https://www.gutenberg.org/cache/epub/1571/pg1571-images.html',

    'Poemas':  'https://www.gutenberg.org/cache/epub/22531/pg22531-images.html'
    }

In [ ]:
def download_text(url):
 response = requests.get(url)
 soup = BeautifulSoup(response.text, 'html.parser')
 text = soup.get_text()
 return text

In [ ]:
MobyDick=download_text(books["Moby Dick"])
WarPeace=download_text(books["War and Peace"])
PridePrejudice=download_text(books["Pride and Prejudice"])
CrimePunish=download_text(books["Crime and Punishment"])
SherlockHomes=download_text(books["The Adventures of Sherlock Holmes"])
Ulysses=download_text(books["Ulysses"])
Odisea=download_text(books["The Odyssey"])
DivinaComedia=download_text(books["The Divine Comedy"])
GreatGatsby=download_text(books["The Great Gatsby"])
Critias=download_text(books["Critias"])
Poemas=download_text(books["Poemas"])

4. Implementar una función para analizar el texto y contar las ocurrencias de palabras
vinculadas con emociones en el texto. Esta función debe:
    - Leer el texto y dividirlo en palabras individuales (tokenización).
    - Asignar a cada palabra su correspondiente etiqueta de parte del discurso (POS tagging) para
    diferenciar entre verbos, sustantivos, adjetivos, etc.
    - Lemmatizar las palabras para reducirlas a su forma base (por ejemplo, "running" a "run").
    - Comparar cada dupla <lemma, POS-tag> con las entradas en el léxico extendido para
    determinar la emoción asociada.
    - Contar las ocurrencias de cada emoción en el texto y generar un informe detallado.

Se ha generado la siguiente función llamada analyze_text_emotions que hace lo siguiente:
- Utilizado la función word_tokenize, se divide el texto en palabras. Esto devuelve un array formado por las palabras.
- Se eliminan las stopwords del texto. La stopword son palabras muy comunes en un idioma pero carecen de valor semántico con son las conjunciones, determinantes,etc. Para ello se descarga y utiliza el corpues stopword de nltk en ingles, para ayudarnos a filtrar estas palabras.
- Se añade su pos_tag a cada palabra tras el filtro .
- Se lematizan todas las palabras, es decir se reduce una palabra a su forma canónica. Para se tienen en cuenta dos problemas:
    - Algunas palabras tienen un pos_tag igual a "''", que suelen ser por fallos de etiquetación del método pos_tag. Esto pasa con la palabra "know" y "noserag", por lo que se vuelven a clasificar para resolverlo.
    - Y no lematizar palabras que tienen un pos_tag igual a "c" y a "x". Esto es debido a que son palabras que carecen de una forma canónica o son palabras desconocidas para el lematizador. Tras hacer la eliminación de las stopwords se ha decidido no eliminar estas palabras, ya que se ha comprobado que pueden tener cierto contexto emocional.
    - Por último, se realiza el conteo de apariciones de sentimientos, que solo se evaluan las palabras que coinciden con el léxico extendido. Se devuelve una lista con el conteo ordenado alfabeticamento por las emociones posibles.


In [ ]:
stop_words = set(stopwords.words('english'))

def analyze_text_emotions(text, emociones_dict):
    # Tokenización
    tokens = word_tokenize(text)

    # Eliminación de stopwords y puntuación
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]

    # POS tagging
    tagged = pos_tag(tokens)

    # Lematización con las etiquetas POS convertidas
    lemmatizer = WordNetLemmatizer()
    lemmatized_words=[]

    for word,tag in tagged:
        if tag=="''":
            tag=pos_tag([word])[0][1]
        if penn_to_wordnet[tag]!="c" and penn_to_wordnet[tag]!="x":
            lemmatized_words.append((lemmatizer.lemmatize(word, pos=penn_to_wordnet[tag]), tag))
        else:
            lemmatized_words.append((word,tag))

    # Contar emociones
    emociones_count = Counter()

    for word, pos in lemmatized_words:
        # Convertimos la tupla (word, pos) en una clave del léxico de emociones
        if (word, pos) in emociones_dict:
            emocion_list = emociones_dict[(word, pos)]
            for emocion in emocion_list:
                emociones_count[emocion] += 1

    return dict(sorted(emociones_count.items()))


In [ ]:
mobyDick_analizado = analyze_text_emotions(MobyDick, emociones_dict)
WarPeace_analizado=analyze_text_emotions(WarPeace, emociones_dict)
PridePrejudice_analizado = analyze_text_emotions(PridePrejudice, emociones_dict)
CrimePunish_analizado = analyze_text_emotions(CrimePunish, emociones_dict)
SherlockHomes_analizado=analyze_text_emotions(SherlockHomes, emociones_dict)
Ulysses_analizado = analyze_text_emotions(Ulysses, emociones_dict)
Odisea_analizado = analyze_text_emotions(Odisea, emociones_dict)
DivinaComedia_analizado = analyze_text_emotions(DivinaComedia, emociones_dict)
GreatGatsby_analizado=analyze_text_emotions(GreatGatsby, emociones_dict)
Critias_analizado = analyze_text_emotions(Critias, emociones_dict)
Poemas_analizado = analyze_text_emotions(Poemas, emociones_dict)

La siguiente función nos muestra los datos analizados a través de un gráfico de barras mostrando la cantidad de cada una de las emociones y un gráfico circular que muestra el porcentaje ocupado de la cantidad de emociones totales. Esto servirá como informe para mostrar los resultados de todas las novelas. El tamaño del libro varía enormemente por lo que la escala difiere entre casi todas las novelas.

In [ ]:
def mostrar_resultado_libro(libro_analizado,nombre):
    # Crear una figura
    fig = go.Figure()

    # Agregar las barras al gráfico
    for emotion, count in libro_analizado.items():
        fig.add_trace(go.Bar(x=[emotion], y=[count], name=emotion))

    # Actualizar el diseño del gráfico
    fig.update_layout(
        title='Conteo de Emociones en '+nombre,
        xaxis=dict(title='Emoción'),
        yaxis=dict(title='Conteo'),
        barmode='group',
        height=500,  # Ajustar la altura de la figura
        width=800    # Ajustar el ancho de la figura
    )

    # Mostrar la figura
    fig.show()

    # Crear una figura para un gráfico de pastel
    fig = go.Figure(data=[go.Pie(labels=list(libro_analizado.keys()), values=list(libro_analizado.values()))])

    # Actualizar el diseño del gráfico
    fig.update_layout(title='Distribución de Emociones en '+nombre,height=500,width=800)

    # Mostrar la figura
    fig.show()

### MobyDick

In [ ]:
mostrar_resultado_libro(mobyDick_analizado,"Moby Dick")

### War and Peace

In [ ]:
mostrar_resultado_libro(Poemas_analizado,"War and Peace")

### Pride and Prejudice

In [ ]:
mostrar_resultado_libro(PridePrejudice_analizado,"Pride and Prejudice")

### Crime and Punishment

In [ ]:
mostrar_resultado_libro(CrimePunish_analizado,"Crime and Punishment")

### Adventures of Sherlock Holmes

In [ ]:
mostrar_resultado_libro(SherlockHomes_analizado,"Sherlock Holmes")

### Ulysses

In [ ]:
mostrar_resultado_libro(Ulysses_analizado,"Ulysses")

### Odyssey

In [ ]:
mostrar_resultado_libro(Odisea_analizado,"Odyssey")

### Divine Comedy

In [ ]:
mostrar_resultado_libro(DivinaComedia_analizado,"Divine Comedy")

### The Great Gatsby

In [ ]:
mostrar_resultado_libro(GreatGatsby_analizado,"Great Gatsby")

### Critias

In [ ]:
mostrar_resultado_libro(Critias_analizado,"Critias")

### Poemas

In [ ]:
mostrar_resultado_libro(Poemas_analizado,"Poemas")

5. Presentar los resultados del análisis de sentimientos en las novelas clásicas. Incluir
estadísticas sobre las emociones más comunes y cualquier patrón interesante que hayas observado.
Considerar cómo visualizar los datos y cómo explicar las conclusiones del análisis solicitado.

La función de a continuación muestra una comparación entre las distintas novelas. Se hace dos comparaciones entre dos grupos de novelas que tienen una cantidad similar de palabras procesadas. El primer grupo formado por las novelas de "Moby Dick", "Crime and Punishment" y "Ulysses" siendo Ulysses una novela con mayor número de palabras. El segundo grupo compara "Pride and Prejudice", "Odyssey" y "Divine comedy".

In [ ]:
def create_multi_book_emotion_chart(emotion_counts_dict):
    fig = go.Figure()

    # Obtener una lista de todas las emociones únicas en todos los libros para asegurar una comparación uniforme
    all_emotions = set()
    for counts in emotion_counts_dict.values():
        all_emotions.update(counts.keys())

    # Procesar cada libro y sus conteos de emociones
    for book, emotion_counts in emotion_counts_dict.items():

        # Asegurar que todas las emociones estén presentes, incluso si no se encontraron en este libro
        for emotion in all_emotions:
            if emotion not in emotion_counts:
                emotion_counts[emotion] = 0

        fig.add_trace(go.Bar(name=book, x=list(all_emotions), y=[emotion_counts[emotion] for emotion in all_emotions]))

    fig.update_layout(
        title='Comparación de Emociones entre Libros',
        xaxis=dict(title='Emoción'),
        yaxis=dict(title='Conteo Ponderado de Emociones'),
        barmode='group'
    )

    fig.show()


In [ ]:
emotion_counts_dict = {
    'Moby Dick': mobyDick_analizado,
    'Crime and Punishment': CrimePunish_analizado,
    'Ulysses': Ulysses_analizado
}

create_multi_book_emotion_chart(emotion_counts_dict)

Se observa que la emoción trust es la emoción más repetida en las tres novelas y la emoción menos repetida disgust. Las emociones son mayores en la novela Ulysses debido a su mayor escala. Entre Crime and punishment y Moby Dick los valores son similares, con diferencias pequeñas entre las distintas emociones, pudiendo ser debibo a temáticas similares entre las novelas.

In [ ]:
emotion_counts_dict = {
    'Pride and Prejudice': PridePrejudice_analizado,
    'Odyssey': Odisea_analizado,
    'Divine Comedy': DivinaComedia_analizado
}

create_multi_book_emotion_chart(emotion_counts_dict)

En este grupo vuelve a destacar la emoción trust, y disgust repite como la de menor aparición en las tres novelas. Por lo demás las emociones tienen mayor diferencia entre novelas que con el grupo anterior. Esto puede ser debido  a que las novelas Moby Dick y Crime and Punishment tienen una mayor similitud entre ellas, transmitiendo emociones similares, al contrario que con las novelas de este grupo.

A continuación vamos a corroborar si trust es la emoción más repetida en todas las novelas, para observar si hay algún problema de sesgo en el léxico que hace que este sentimiento salga favorecido en las estadísticas.

In [ ]:
def create_emotion_dominance_chart(emotion_counts_dict, texto="Predominante", filtro=[], funcion=max):
    books = list(emotion_counts_dict.keys())
    dominant_emotions = []
    dominant_counts = []

    for _, emotion_counts in emotion_counts_dict.items():

        filtered_emotion_counts = {emotion: count for emotion, count in emotion_counts.items() if emotion not in filtro}

        if not filtered_emotion_counts:
            dominant_emotions.append("N/A")
            dominant_counts.append(0)
            continue

        # Encontrar la emoción dominante y su conteo
        dominant_emotion, dominant_count = funcion(filtered_emotion_counts.items(), key=lambda item: item[1])
        dominant_emotions.append(f"{dominant_emotion} ({dominant_count})")
        dominant_counts.append(dominant_count)

    fig = go.Figure(go.Bar(
        y=books,
        x=dominant_counts,
        text=dominant_emotions,
        orientation='h',
        marker=dict(color=dominant_counts, coloraxis="coloraxis")
    ))

    fig.update_layout(
        title='Emoción '+texto+' en Cada Libro',
        xaxis=dict(title='Conteo de la Emoción '+texto),
        yaxis=dict(title='Libros'),
        coloraxis=dict(colorscale='Viridis'),
        showlegend=False
    )

    fig.show()


In [ ]:
emotion_counts_dict = {
    "MobyDick": mobyDick_analizado,
    'War': WarPeace_analizado,
    'Pride and Prejudice':PridePrejudice_analizado,
    'CrimePunish':CrimePunish_analizado,
    'SherlockHomes': SherlockHomes_analizado,
    'Ulysses':Ulysses_analizado,
    'Odisea':Odisea_analizado,
    'Divina': DivinaComedia_analizado,
    'Great gatsby': GreatGatsby_analizado,
    'Critias':Critias_analizado,
    'Poema': Poemas_analizado
}
create_emotion_dominance_chart(emotion_counts_dict)

Se corrobora que la emoción predominante en todas las novelas es trust, más adelante se comprueba y analiza si es debido a algún sesgo del léxico extendido.

In [ ]:
emociones_count = {}
for emocion_lista in emociones_dict.values():
    for emocion in emocion_lista:
        if emocion in emociones_count:
            emociones_count[emocion] += 1
        else:
            emociones_count[emocion] = 1

In [ ]:
mostrar_resultado_libro(emociones_count,"lexico extendido")

El gráfico muestra la distribución de emociones en el léxico extendido, observando que las dos emociones con más presencia son fear y trust. Se demuestra que sí existe un sesgo, ya que no estan igual de representadas todas las emociones. Y pudiendo ser una de las razones por la que trust tiene tanta predominancia respecto al resto de emociones, pero con la emoción fear que tiene una mayor presencia en el léxico no ocurre lo mismo.

Esto simplemente puede ser debido a que la temática trust era mucho más favorable que fear en las novelas evaluadas, pudiendo indicar la tendencia de esos escritores por esa emoción. El siguiente gráfico muestra la predominancia de las emociones eliminando trust para observar cual es la tendencia de los escritores.

In [ ]:
create_emotion_dominance_chart(emotion_counts_dict,filtro=["trust"])

Ahora se observa una predominancia de la emoción anticipation, pero no al nivel que tenía la emoción trust. También se observa la aparición de las emociones joy y sadness. Ahora se prueba eliminando las emociones trust y anticipation, para seguir observando la tendencia de las novelas.

In [ ]:
create_emotion_dominance_chart(emotion_counts_dict,filtro=["trust","anticipation"])

Ahora se observa una predominancia tanto de los sentimientos joy y fear.
Con estas demostraciones se ha determinado que la tendencia de las temáticas de los autores en las obras evaluadas son trust, anticipation, joy y fear teniendo como valor outcast sadness de la novela de poemas. Estas emociones ordenadas de mayor a menor aparición.

También se demuestra que aún habiendo una diferencia de representación de las emociones en el léxico extendido, no tienen un efecto tan negativo en el análisis, debido a que dos de las emociones, como lo son joy y anticipation, no tienen tanta relevancia en el léxico y aún así tienen una alta predominancia.

En cuanto a las emociones menos representativas se observa lo siguiente:

In [ ]:
create_emotion_dominance_chart(emotion_counts_dict,texto="Menos Predominante",funcion=min)

In [ ]:
create_emotion_dominance_chart(emotion_counts_dict,texto="Menos Predominante",filtro=["disgust"],funcion=min)

Y en cuanto a las emociones menos representativas son disgust y surprise. Estas dos emociones son las que menos tendencia tienden a utilizar los autores de las novelas evaluadas.